<a href="https://colab.research.google.com/github/Pratyushk2003/healthcare/blob/main/DiseasePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install catboost
%pip install scispacy
%pip install spacy
%pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_md-0.5.1.tar.gz
%pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 110.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 24.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 10.5 MB/s eta 0:00:00
  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
  Created wheel for nmslib: filename=nmslib-2.1.1-cp310-cp310-linux_x86_64.whl size=13572299 sha256=b8ba139a3cb81ff1b45bfbf85ed763bd62fbe8d9e11ef0143cd975da51e2c2f0
  Stored in directory: /root/.cache/pip/wheels/21/1a/5d/4cc754a5b1a88405cad184b76f823897a63a8d19afcd4b9314
Successfully built nmslib
 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/content/main.csv")

In [ ]:
df.head()

,frequency,UMLS:C0457096_yellow sputum,UMLS:C0425560_cardiovascular finding,UMLS:C0020440_hypercapnia,UMLS:C0581912_heavy feeling,UMLS:C0002416_ambidexterity,UMLS:C0521516_polymyalgia,UMLS:C0677500_stinging sensation,UMLS:C0392680_shortness of breath,UMLS:C0030252_palpitation,...,UMLS:C0030193_pain,UMLS:C0239832_numbness of hand,UMLS:C0277977_Murphy's sign,UMLS:C0740844_air fluid level,UMLS:C0026827_muscle hypotonia,UMLS:C0010200_cough,UMLS:C0043094_weight gain,UMLS:C0600142_hot flush,UMLS:C0312422_blackout,label
0,3363.0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,UMLS:C0020538_hypertensive disease
1,1421.0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,UMLS:C0011847_diabetes
2,1337.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,UMLS:C0011570_depression mental^UMLS:C0011581_...
3,1284.0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,UMLS:C0010054_coronary arteriosclerosis^UMLS:C...
4,1029.0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,UMLS:C0032285_pneumonia


In [ ]:
mod_df = df.loc[df.index.repeat(df['frequency'])].reset_index(drop=True)

In [ ]:
from sklearn.utils import class_weight
from sklearn.preprocessing import LabelEncoder

label = mod_df['label']
encoder = LabelEncoder()
encoded_label = encoder.fit_transform(label)

class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(encoded_label), y=encoded_label)


In [ ]:
encoded_label.shape

(33724,)

In [ ]:
mod_df.drop(['frequency', 'label', 'UMLS:C0032739_'], axis=1, inplace=True)

In [ ]:
mod_df.head()

,UMLS:C0457096_yellow sputum,UMLS:C0425560_cardiovascular finding,UMLS:C0020440_hypercapnia,UMLS:C0581912_heavy feeling,UMLS:C0002416_ambidexterity,UMLS:C0521516_polymyalgia,UMLS:C0677500_stinging sensation,UMLS:C0392680_shortness of breath,UMLS:C0030252_palpitation,UMLS:C0020621_hypokalemia,...,UMLS:C0233071_para 2,UMLS:C0030193_pain,UMLS:C0239832_numbness of hand,UMLS:C0277977_Murphy's sign,UMLS:C0740844_air fluid level,UMLS:C0026827_muscle hypotonia,UMLS:C0010200_cough,UMLS:C0043094_weight gain,UMLS:C0600142_hot flush,UMLS:C0312422_blackout
0,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(mod_df, encoded_label, test_size=0.2, random_state=42)

In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
import joblib

model = CatBoostClassifier(iterations=18, task_type="GPU", devices='0' ,learning_rate=0.1, depth=6, loss_function='MultiClass', eval_metric='Accuracy', auto_class_weights='Balanced')
# cv = cross_val_score(model, mod_df, encoded_label, cv=5, scoring='accuracy')
model.fit(X_train, y_train, eval_set=(X_test, y_test), use_best_model=True, verbose=True)

# Save the model to a file
joblib.dump(model, 'model.pkl')

0:	learn: 0.0671642	test: 0.0624842	best: 0.0624842 (0)	total: 272ms	remaining: 4.63s
1:	learn: 0.1268657	test: 0.1179046	best: 0.1179046 (1)	total: 528ms	remaining: 4.23s
2:	learn: 0.1716418	test: 0.1654432	best: 0.1654432 (2)	total: 756ms	remaining: 3.78s
3:	learn: 0.2164179	test: 0.2133833	best: 0.2133833 (3)	total: 974ms	remaining: 3.41s
4:	learn: 0.2611940	test: 0.2576944	best: 0.2576944 (4)	total: 1.16s	remaining: 3.02s
5:	learn: 0.3059702	test: 0.3031593	best: 0.3031593 (5)	total: 1.34s	remaining: 2.68s
6:	learn: 0.3507463	test: 0.3446787	best: 0.3446787 (6)	total: 1.5s	remaining: 2.35s
7:	learn: 0.3955224	test: 0.3922079	best: 0.3922079 (7)	total: 1.67s	remaining: 2.09s
8:	learn: 0.4477612	test: 0.4413776	best: 0.4413776 (8)	total: 1.83s	remaining: 1.83s
9:	learn: 0.4925373	test: 0.4910825	best: 0.4910825 (9)	total: 1.97s	remaining: 1.58s
10:	learn: 0.5373134	test: 0.5318031	best: 0.5318031 (10)	total: 2.08s	remaining: 1.32s
11:	learn: 0.6492537	test: 0.6419997	best: 0.6419997 

['model.pkl']

In [ ]:
import en_core_sci_md
nlp_md = en_core_sci_md.load()


In [ ]:
def get_entities(text):
    entity = nlp_md(text)
    return [(ent.text, ent.label_) for ent in entity.ents]

In [ ]:
input_text = input()
entities = get_entities(input_text)
entities

chest pain


[('chest pain', 'ENTITY')]

In [ ]:
# get the similarity between 2 words

# from scipy import spatial

# def get_similarity(word1, word2):
#     return 1 - spatial.distance.cosine(nlp_md(word1).vector, nlp_md(word2).vector)

from difflib import SequenceMatcher

def get_similarity(w1, w2):
  return SequenceMatcher(None, w1, w2).ratio()

In [ ]:
features = mod_df.columns
test_input = pd.DataFrame(columns=features)
test_input.loc[len(test_input)] = np.nan

for (ent, _) in entities:
  max_feature = []
  max_similarity = 0
  for feature in features:
    feature_name = feature.split("_")[1]
    sim = get_similarity(ent, feature_name)
    if max_similarity < sim:
      max_similarity = sim
      max_feature = feature
  test_input[max_feature] = 1
test_input = test_input.fillna(0)

In [ ]:
test = test_input.loc[:, test_input.any()]
print(test)

   UMLS:C0234253_rest pain
0                        1


In [ ]:
#@title
test_input = pd.DataFrame(columns=features)
test_input.loc[len(test_input)] = [0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]


In [ ]:
pred = model.predict(test_input)

encoder.inverse_transform(pred[0])

array(['UMLS:C0020538_hypertensive disease'], dtype=object)

In [ ]:
from google.colab import files

files.download('model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>